Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [32]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [33]:
labels = [
    {'name':'Hello', 'id':1}, 
    {'name':'Yes', 'id':2},
    {'name':'No', 'id':3},
    {'name':'Thanks', 'id':4},
    {'name':'I Love You', 'id':5},
]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [34]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Traceback (most recent call last):
  File "Tensorflow/scripts/generate_tfrecord.py", line 168, in <module>
    tf.app.run()
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\platform\app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\absl\app.py", line 303, in run
    _run_main(main, args)
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\absl\app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "Tensorflow/scripts/generate_tfrecord.py", line 158, in main
    tf_example = create_tf_example(group, path)
  File "Tensorflow/scripts/generate_tfrecord.py", line 132, in create_tf_example
    classes.append(class_text_to_int(row['class']))
  File "Tensorflow/scripts/generate_tfrecord.py", line 101, in class_text_to_int
    return label_map_dict[row_label]
KeyError: 'Good Bye'
Traceba

# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [35]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [36]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [37]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [38]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!copy {PRETRAINED_MODEL_PATH+'\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config'} {MODEL_PATH+'\\'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


The syntax of the command is incorrect.


# 5. Update Config For Transfer Learning

In [39]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [40]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [41]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [42]:
config

{'model': ssd {
   num_classes: 5
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [43]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [44]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [46]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [47]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [48]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [49]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-13')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [50]:
import cv2 
import numpy as np
import torch
from torch_geometric.data import Data
from torchvision import ops as box_utils

In [51]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [25]:
cap.release()

NameError: name 'cap' is not defined

In [52]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [53]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

Exception ignored in: <bound method ScopedTFGraph.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFGraph object at 0x0000015268D66208>>
Traceback (most recent call last):
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
AttributeError: deleter


COBA YANG INI

In [53]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    ssd_features = detect_fn(input_tensor)
    
    num_detections = int(ssd_features.pop('num_detections'))
    ssd_features = {key: value[0, :num_detections].numpy()
                  for key, value in ssd_features.items()}
    ssd_features['num_detections'] = num_detections

    # detection_classes should be ints.
    ssd_features['detection_classes'] = ssd_features['detection_classes'].astype(np.int64)

    detections = detect_fn(input_tensor)
    valid_indices = np.where(detections['detection_scores'] > score_threshold)[0]
    valid_boxes = detections['detection_boxes'][valid_indices]
    iou = box_utils.bbox_overlaps(valid_boxes, valid_boxes)
    edge_mask = iou > iou_threshold
    edge_indices = np.argwhere(edge_mask)
    edge_index = torch.from_numpy(edge_indices)

    # Create graph object
    graph = Data(x=ssd_features, edge_index=edge_index)
    
    # Pass the graph data to your GCN model
    graph_features = gcn(graph)
    
    # Concatenate the SSD features with the GCN features
    concatenated_features = torch.cat((ssd_features, graph_features), axis=-1)

    # Create a fully connected layer to classify the object
    fc = torch.nn.Linear(concatenated_features.shape[-1], num_classes)

    # Pass the concatenated features through the fully connected layer
    logits = fc(concatenated_features)

    # Apply softmax on the logits to get the class probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Get the class with the highest probability
    class_id = torch.argmax(probabilities)

    # Draw the predicted class on the image
    cv2.putText(image_np, class_names[class_id], (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Show the image
    cv2.imshow('object detection',  cv2.resize(image_np, (800, 600)))

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break    

ValueError: in user code:

    <ipython-input-41-b0a77f174d1b>:11 detect_fn  *
        image, shapes = detection_model.preprocess(image)
    c:\Users\ACER\AppData\Local\Programs\Python\Python36\lib\site-packages\object_detection\meta_architectures\ssd_meta_arch.py:485 preprocess  *
        normalized_inputs, self._image_resizer_fn)
    c:\Users\ACER\AppData\Local\Programs\Python\Python36\lib\site-packages\object_detection\utils\shape_utils.py:492 resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    c:\Users\ACER\AppData\Local\Programs\Python\Python36\lib\site-packages\object_detection\utils\shape_utils.py:246 static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    c:\Users\ACER\AppData\Local\Programs\Python\Python36\lib\site-packages\object_detection\core\preprocessor.py:3330 resize_image  *
        new_image = tf.image.resize_images(
    c:\Users\ACER\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\pyth

In [33]:
    ssd_features = detect_fn(input_tensor)
    
    valid_indices = np.where(detections['detection_scores'] > score_threshold)[0]
    valid_boxes = detections['detection_boxes'][valid_indices]
    iou = box_utils.bbox_overlaps(valid_boxes, valid_boxes)
    edge_mask = iou > iou_threshold
    edge_indices = np.argwhere(edge_mask)
    edge_index = torch.from_numpy(edge_indices)

    
    # Create graph object
    graph = Data(x=ssd_features, edge_index=edge_index)
    
    # Pass the graph data to your GCN model
    graph_features = gcn(graph)
    
    # Concatenate the SSD features with the GCN features
    concatenated_features = torch.cat((ssd_features, graph_features), axis=-1)

    # Create a fully connected layer to classify the object
    fc = torch.nn.Linear(concatenated_features.shape[-1], num_classes)

    # Pass the concatenated features through the fully connected layer
    logits = fc(concatenated_features)

    # Apply softmax on the logits to get the class probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Get the class with the highest probability
    class_id = torch.argmax(probabilities)

    # Draw the predicted class on the image
    cv2.putText(image_np, class_names[class_id], (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Show the image
    cv2.imshow('Aplikasi Deteksi Bahasa Isyarat',  cv2.resize(image_np, (800, 600)))

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break    

SyntaxError: 'break' outside loop (<ipython-input-33-941b7d22687c>, line 46)

In [37]:
detections = detect_fn(input_tensor)

NameError: name 'detect_fn' is not defined

In [ ]:
from matplotlib import pyplot as plt